In [1]:
import datetime as dt
import difflib
import json
import math
import os
import string
import sys
from datetime import timedelta
from random import sample

import folium
import matplotlib as mpl
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import pyreadstat
import seaborn as sns
import statsmodels.tools.tools as sm
import statsmodels.base as sb
from folium.features import CustomIcon
from folium.plugins import FastMarkerCluster, HeatMap, MarkerCluster
from linearmodels.panel.model import PooledOLS
from scipy.signal import savgol_filter
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tools.eval_measures import aic, rmse
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
import yfinance as yf
from yahoofinancials import YahooFinancials
import pickle
import array_to_latex as a2l
import markdown
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

In [4]:
data = pd.read_csv(
    "./markettrends0.csv",
    dtype={
        "state": "str",
        "sa3_name16": "str",
        "sa4_name16": "str",
        "postcode": "str",
        "state": "str",
        "property_type": "str",
    },
)
ndata = data.fillna(
    {"Volume of new rental listings (1 month)": 0, "Volume of sales (1 month)": 0}
).dropna(subset=["postcode"])
ndatahouses = ndata[:][ndata.property_type == "Houses"]
ndataunits = ndata[:][ndata.property_type == "Units"]

In [ ]:
UniqueNames_sa4 = ndatahouses.sa4_name16.unique()[:-1]

In [16]:
ndatahouses.sa4_name16.unique()

array(['Darwin', 'Northern Territory - Outback',
       'Sydney - City and Inner South', 'Sydney - Eastern Suburbs',
       'Sydney - Inner West', 'Sydney - North Sydney and Hornsby',
       'Sydney - Northern Beaches', 'Sydney - Ryde',
       'Sydney - Parramatta', 'Sydney - Baulkham Hills and Hawkesbury',
       'Sydney - Blacktown', 'Sydney - South West',
       'Sydney - Outer South West', 'Sydney - Inner South West',
       'Sydney - Sutherland', 'Central Coast',
       'Newcastle and Lake Macquarie', 'Hunter Valley exc Newcastle',
       'Mid North Coast', 'New England and North West',
       'Far West and Orana', 'Coffs Harbour - Grafton',
       'Richmond - Tweed', 'Illawarra',
       'Southern Highlands and Shoalhaven', 'Capital Region', 'Riverina',
       'Australian Capital Territory', 'Murray', 'Central West',
       'Sydney - Outer West and Blue Mountains', 'Melbourne - Inner',
       'Melbourne - West', 'Melbourne - North West',
       'Melbourne - North East', 'Melbourne

In [ ]:
year = list(range(2001, 2021))
DataFrameDict_HILDAp = {elem: pd.DataFrame for elem in year}
for key in DataFrameDict_HILDAp.keys():
    order = key - 2001
    ini = string.ascii_lowercase[order]
    df, meta = pyreadstat.read_dta(f"Rperson_{ini}200u.dta")
    df.columns = df.columns.str[1:]
    DataFrameDict_HILDAp[key] = df
DataFrameDict_HILDA_cw = {elem: pd.DataFrame for elem in year}
for key in DataFrameDict_HILDA_cw.keys():
    order = key - 2001
    ini = string.ascii_lowercase[order]
    df, meta = pyreadstat.read_dta(f"Combined_{ini}200u.dta")
    df.columns = df.columns.str[1:]
    DataFrameDict_HILDA_cw[key] = df
DataFrameDict_HILDA = {elem: pd.DataFrame for elem in year}
for key in DataFrameDict_HILDA.keys():
    order = key - 2001
    ini = string.ascii_lowercase[order]
    df, meta = pyreadstat.read_dta(f"Household_{ini}200u.dta")
    df.columns = df.columns.str[1:]
    DataFrameDict_HILDA[key] = df    

In [ ]:
list_postcodes = DataFrameDict_HILDA[2001]["hhpcode"]
for key in DataFrameDict_HILDA.keys():
    list_ny = DataFrameDict_HILDA[key]["hhpcode"]
    list_postcodes = list(set(list_postcodes).intersection(list_ny))
count_dict = dict()
for key in DataFrameDict_HILDA.keys():
    for item in DataFrameDict_HILDA[key]["hhpcode"].unique():
        if item in count_dict:
            count_dict[item] += 1
        else:
            count_dict[item] = 1
df_allpostcode = pd.DataFrame(count_dict.items())
HILDAallpc = df_allpostcode[0].values
df_retainedpc = df_allpostcode.drop(df_allpostcode[df_allpostcode[1] < 20].index)